In [2]:
import torch

https://lernapparat.de/jit-optimization-intro/


In [578]:
class mlp(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(10, 3),
            torch.nn.Softplus(),
            torch.nn.Linear(3,10),
            torch.nn.Softplus(),
        )
    
    def forward(self, x):
        return self.net(x)
    
class mlp_jac(torch.nn.Module):
    def __init__(self, mlp):
        super().__init__()        
        self.mlp = mlp.net

        self.linear1 = self.mlp[0]
        self.linear2 = self.mlp[2]
        
        self.linear1_weight = self.linear1.weight
        self.linear1_bias   = self.linear1.bias
        
        self.linear2_weight = self.linear2.weight
        self.linear2_bias   = self.linear2.bias

    def forward(self,x):
        
        jac1, out1 = self.jacobian_onelayer(self.linear1_weight, self.linear1_bias, x)
        jac2, out2 = self.jacobian_onelayer(self.linear2_weight, self.linear2_bias, out1)
        
        jacs = torch.bmm(jac2.permute(2,0,1),jac1.permute(2,0,1))
        return jacs
#         jacs = torch.einsum("xyb, yzb-> xzb", output_jac2, output_jac1)
        
        return jac1, jac2
    
    def jacobian_onelayer(self, W, b, x):
        lin_out = F.linear(x, W, b)
        # assume a softplux activation
        out     = F.softplus(lin_out)
        # for sigmoid only!!!
        dz_dx = torch.sigmoid(lin_out)
        jac = dz_dx.T.unsqueeze(1)*W.unsqueeze(-1)
        return jac, out

In [579]:
model     = mlp().double()
model_jac = mlp_jac(model)

In [580]:
x = torch.randn(2,10, dtype=torch.float64)

In [592]:
jac_out = torch.autograd.functional.jacobian(model,x)

assert (model_jac(x)[0] - jac_out[0,:,0,:]).abs().sum()  < 1e-15
assert (model_jac(x)[1] - jac_out[1,:,1,:]).abs().sum()  < 1e-15

In [594]:
sm = torch.jit.script(model)
sm.save("sample_mlp.pt")

In [595]:
sm_jac = torch.jit.script(model_jac)
sm_jac.save("sample_mlp_jac.pt")

In [597]:
jac_output = model_jac(x)

In [598]:
jac_output

tensor([[[ 7.3412e-02,  9.6209e-03,  1.1615e-03, -1.9757e-03, -5.5300e-02,
          -1.5132e-02,  5.0901e-02, -5.1170e-02, -4.9526e-02,  2.9634e-02],
         [-7.1423e-03, -1.1505e-02, -3.4237e-03,  3.0132e-03, -4.7993e-03,
          -5.8923e-03,  1.2469e-02,  6.7605e-03, -1.0347e-02, -6.1251e-03],
         [-1.4138e-02, -2.1848e-02, -6.4179e-03,  7.6606e-03, -8.4621e-03,
          -1.3007e-02,  2.5023e-02,  1.2197e-02, -1.9448e-02, -1.1716e-02],
         [-9.7071e-03, -1.9917e-02, -5.2517e-03,  2.6016e-02, -9.0773e-03,
          -3.2255e-02,  4.3957e-02, -1.0942e-03, -2.3346e-02, -8.3005e-03],
         [-3.4236e-02, -1.6066e-02, -3.8222e-03,  1.3716e-02,  1.5369e-02,
          -1.0943e-02,  4.6684e-03,  2.0677e-02,  5.0139e-03, -1.6747e-02],
         [ 1.9344e-02, -6.1773e-04, -1.5915e-04,  1.4941e-02, -1.6505e-02,
          -2.1146e-02,  3.2649e-02, -2.0695e-02, -1.9784e-02,  7.7824e-03],
         [-1.1096e-02, -4.0391e-03, -3.7554e-04,  1.8053e-02,  7.1572e-03,
          -1.6977e-

In [165]:
from torch.autograd.functional import jacobian

In [169]:
# model_jac = mlp_jac(model)?
output = model(x)

In [172]:
jac_out = torch.autograd.functional.jacobian(model,x)

In [524]:
torch.autograd.grad?

Signature:
torch.autograd.grad(
    outputs: Union[torch.Tensor, Sequence[torch.Tensor]],
    inputs: Union[torch.Tensor, Sequence[torch.Tensor]],
    grad_outputs: Union[torch.Tensor, Sequence[torch.Tensor], NoneType] = None,
    retain_graph: Optional[bool] = None,
    create_graph: bool = False,
    only_inputs: bool = True,
    allow_unused: bool = False,
) -> Tuple[torch.Tensor, ...]
Docstring:
Computes and returns the sum of gradients of outputs with respect to
the inputs.

``grad_outputs`` should be a sequence of length matching ``output``
containing the "vector" in Jacobian-vector product, usually the pre-computed
gradients w.r.t. each of the outputs. If an output doesn't require_grad,
then the gradient can be ``None``).

If ``only_inputs`` is ``True``, the function will only return a list of gradients
w.r.t the specified inputs. If it's ``False``, then gradient w.r.t. all remaining
leaves will still be computed, and will be accumulated into their ``.grad``
attribute.

.. note:

In [3]:
from typing import List, Optional
from torch import Tensor
class mlp_jac(torch.nn.Module):
    def __init__(self, mlp):
        super().__init__()
        self.mlp = mlp
    def forward(self, x: torch.Tensor):
        returns = torch.jit.annotate(List[Tensor], [])
#         t = torch.jit.annotate(Tensor,torch.zeros(2,10))
        out = self.mlp(x)
        j = 0
        for row in out:
            for i in row:
                t = torch.autograd.grad([i],[x], retain_graph=True)[0]
                returns += [t[j]]
            j += 1
        return torch.stack(returns).reshape(2,10,10)

In [4]:
torch.autograd.grad([out[0][0], out[0][1] ], [x], retain_graph=True)[0]

NameError: name 'out' is not defined

In [5]:
out = model(x)
torch.autograd.grad([out[0][0]], [x], retain_graph=True)

NameError: name 'model' is not defined

In [6]:
torch.autograd.grad([out[0][1]], [x], retain_graph=True)

NameError: name 'out' is not defined

In [7]:
model = mlp().double()
jac_model = mlp_jac(model)

NameError: name 'mlp' is not defined

In [521]:
x = torch.randn(2,10, dtype=torch.float64, requires_grad=True)
out_list = jac_model(x.double())

In [522]:
out_list

tensor([[[-3.2084e-04,  2.2529e-03,  4.0516e-03, -3.1866e-03,  7.0165e-03,
          -4.5932e-04,  6.4646e-04,  4.2147e-03,  3.0470e-03, -3.9829e-03],
         [ 8.7213e-03, -8.0255e-03, -1.4579e-03,  8.7490e-03, -2.4656e-03,
           1.3933e-04,  1.6632e-04,  2.3817e-03,  2.3777e-04,  1.0260e-02],
         [ 2.1913e-03, -3.0560e-03, -4.8507e-03,  7.8950e-03, -9.5200e-03,
           2.2615e-03,  2.5245e-03, -3.0884e-03, -6.3929e-03,  7.6157e-03],
         [-7.3053e-03,  1.0046e-05, -3.8962e-03, -1.2697e-02, -2.6810e-03,
          -5.7822e-03, -1.2670e-02, -1.1646e-02,  6.8951e-03, -7.5945e-03],
         [-3.7289e-03,  1.1122e-03,  1.2331e-03, -9.8001e-03,  4.6734e-03,
          -4.0161e-03, -7.2494e-03, -2.9407e-03,  7.2130e-03, -7.1863e-03],
         [ 7.8363e-03, -4.4045e-03,  5.7665e-03,  1.3763e-03,  1.0688e-02,
          -1.6565e-03, -5.2603e-04,  9.1066e-03,  7.3450e-03,  2.2714e-03],
         [-1.2230e-02,  1.0107e-02, -5.4762e-04, -1.0150e-02, -1.1521e-03,
           2.8002e-

In [523]:
model = torch.jit.script(jac_model)

RuntimeError: 
'Optional[Tensor]' object is not subscriptable:
  File "/tmp/ipykernel_33949/1924489597.py", line 17
            for i in row:
                t = torch.autograd.grad([i],[x], retain_graph=True)[0]
                returns += [t[j]]
                            ~~~~ <--- HERE
            j += 1
        return torch.stack(returns).reshape(2,10,10)


In [420]:
x = torch.randn(2,10, dtype=torch.float64, requires_grad=True)
model(x.double())

[tensor([[ 0.0025,  0.0035,  0.0061,  0.0022,  0.0050,  0.0007,  0.0060,  0.0031,
          -0.0022,  0.0062],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]], dtype=torch.float64),
 tensor([[-0.0052, -0.0027,  0.0004, -0.0053, -0.0032, -0.0021,  0.0029, -0.0058,
          -0.0030,  0.0079],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]], dtype=torch.float64),
 tensor([[-0.0039,  0.0084, -0.0048, -0.0064, -0.0046,  0.0099, -0.0076, -0.0013,
          -0.0001, -0.0021],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]], dtype=torch.float64),
 tensor([[ 0.0040, -0.0047,  0.0011,  0.0057,  0.0029, -0.0051,  0.0016,  0.0026,
           0.0021, -0.0041],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]], dtype=torch.float64),
 tensor([[ 0.0069, -

In [8]:
from collections import OrderedDict

In [208]:
class onelayer_mlp(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            OrderedDict([
                ('linear1', torch.nn.Linear(3, 8)),
                ('activation1', torch.nn.Sigmoid()),
                ('linear2', torch.nn.Linear(8, 5)),
                ('activation2', torch.nn.Sigmoid()),
            ])
#             torch.nn.Linear(3,10),
#             torch.nn.Sigmoid(),
        )
    
    
    def forward(self, x):
        return self.net(x)

In [538]:
class mlp_jac(torch.nn.Module):
    def __init__(self, mlp):
        super().__init__()        
        self.mlp = mlp.net

        self.linear1 = self.mlp[0]
        self.linear2 = self.mlp[2]
        
        self.linear1_weight = self.linear1.weight
        self.linear1_bias   = self.linear1.bias
        
        self.linear2_weight = self.linear2.weight
        self.linear2_bias   = self.linear2.bias

    def forward(self,x):
        
        jac1, out1 = self.jacobian_onelayer(self.linear1_weight, self.linear1_bias, x)
        jac2, out2 = self.jacobian_onelayer(self.linear2_weight, self.linear2_bias, out1)
        
        jacs = torch.bmm(jac2.permute(2,0,1),jac1.permute(2,0,1))
        return jacs
#         jacs = torch.einsum("xyb, yzb-> xzb", output_jac2, output_jac1)
        
        return jac1, jac2
    
    def jacobian_onelayer(self, W, b, x):
        lin_out = F.linear(x, W, b)
        out     = torch.sigmoid(lin_out)
        # for sigmoid only!!!
        dz_dx = out* (1 - out)
        jac = dz_dx.T.unsqueeze(1)*W.unsqueeze(-1)
        return jac, out

In [539]:
mlp1     = onelayer_mlp()
mlp1_jac = mlp_jac(mlp1)

In [540]:
x = torch.randn(2,3)

In [541]:
jac_true = torch.autograd.functional.jacobian(mlp1,x)
jac_true[0,:,0,:]

tensor([[-0.0072, -0.0018,  0.0078],
        [ 0.0020,  0.0017,  0.0005],
        [-0.0027,  0.0145, -0.0197],
        [-0.0045, -0.0025,  0.0069],
        [-0.0049,  0.0053, -0.0059]])

In [542]:
import torch.nn.functional as F

In [543]:

# out1 = (mlp1.net(x))
# w = mlp1.net[0].weight
# do_dz = out1* (1 - out1)
# jac = do_dz.T.unsqueeze(1)*w.unsqueeze(-1)

In [544]:
jac.shape

torch.Size([8, 3, 2])

In [545]:
mlp1_jac = mlp_jac(mlp1)
output_jac = mlp1_jac(x)

In [550]:
output_jac.shape

torch.Size([2, 5, 3])

In [547]:
aaa = torch.jit.script(mlp1_jac)

In [549]:
aaa.save("mlp1_jac.ptc")

In [552]:
mlp_script = torch.jit.load("sample_mlp.pt")

In [560]:
mlp_jac(mlp_script)

NotImplementedError: 

In [566]:
mlp_script._parameters.__dict__['_c']